***GENERATED CODE FOR xgboost PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'ChargeDetailID0'}], 'feature': 'ChargeDetailID0', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3042', 'mean': '', 'stddev': '', 'min': '0026482c-a666-4d83-8387-1febcab99514', 'max': 'fff4b81e-a247-4ec5-ab2f-edb605619ac3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID0'}, {'transformation_label': 'String Indexer', 'feature_label': 'ChargeDetailID0'})
        transformationDF = transformationDF.drop('ChargeDetailID0')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'RVUModifier'}], 'feature': 'RVUModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '26.0', 'stddev': '0.0', 'min': '26', 'max': 'TC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RVUModifier'}, {'transformation_label': 'String Indexer', 'feature_label': 'RVUModifier'})
        transformationDF = transformationDF.drop('RVUModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'Gender'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'transformation_label': 'String Indexer', 'feature_label': 'Gender'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'PayerName'}], 'feature': 'PayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': 'AARPMedicareComplete', 'max': 'WorkersComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerName'}, {'transformation_label': 'String Indexer', 'feature_label': 'PayerName'})
        transformationDF = transformationDF.drop('PayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'PayerCategory'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'WorkComp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'transformation_label': 'String Indexer', 'feature_label': 'PayerCategory'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'PayerGroup'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'OtherGovernment', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerGroup'}, {'transformation_label': 'String Indexer', 'feature_label': 'PayerGroup'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'SourceKey58'}], 'feature': 'SourceKey58', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR96', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SourceKey58'}, {'transformation_label': 'String Indexer', 'feature_label': 'SourceKey58'})
        transformationDF = transformationDF.drop('SourceKey58')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'ReasonDate'}], 'feature': 'ReasonDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': '01/05/2020', 'max': '31/03/2020', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDate'}, {'transformation_label': 'String Indexer', 'feature_label': 'ReasonDate'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'FacilityID63'}], 'feature': 'FacilityID63', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': '087613d0-c7f3-4bcd-900a-abbc6ee3bdad', 'max': 'fde71f17-372e-4cde-95a7-d8b9b45a015a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityID63'}, {'transformation_label': 'String Indexer', 'feature_label': 'FacilityID63'})
        transformationDF = transformationDF.drop('FacilityID63')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'DepartmentID'}], 'feature': 'DepartmentID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3042', 'mean': '', 'stddev': '', 'min': '0a414089-7b3a-4378-ba12-23a2d7272bcf', 'max': 'fb68b47d-bad4-42c2-9ad0-9816864ea30a', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentID'}, {'transformation_label': 'String Indexer', 'feature_label': 'DepartmentID'})
        transformationDF = transformationDF.drop('DepartmentID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run xgboostHooks.ipynb
try:
	#sourcePreExecutionHook()

	dbfs = DBFSConnector.fetch(spark, "{'url': '/Whitespace/whitespace_Classification_denails.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(dbfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run xgboostHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(dbfs,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "ChargeDetailID0"}], "feature": "ChargeDetailID0", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "0026482c-a666-4d83-8387-1febcab99514", "max": "fff4b81e-a247-4ec5-ab2f-edb605619ac3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3042", "mean": "233.29", "stddev": "573.23", "min": "4.0", "max": "23900.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3042", "mean": "84.55", "stddev": "184.2", "min": "-566.3", "max": "3368.38", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "1733.23", "stddev": "1207.04", "min": "1", "max": "4079", "missing": "0"}, "updatedLabel": "CPTCodeID3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "29769.92", "stddev": "22492.44", "min": "13501", "max": "94911", "missing": "0"}, "updatedLabel": "DiagnosisCodeID2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "38125.66", "stddev": "20760.23", "min": "13571", "max": "94819", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "RVUModifier"}], "feature": "RVUModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "26.0", "stddev": "0.0", "min": "26", "max": "TC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RVUModifier"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "Gender"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EnterpriseID18", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "EnterpriseID18"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "PayerName"}], "feature": "PayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "AARPMedicareComplete", "max": "WorkersComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerName"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "PayerCategory"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "Aetna", "max": "WorkComp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "PayerGroup"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "Aetna", "max": "OtherGovernment", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerGroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SpecialityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "35.98", "stddev": "30.4", "min": "-1", "max": "120", "missing": "0"}, "updatedLabel": "SpecialityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TaxonomyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "352.15", "stddev": "167.51", "min": "-1", "max": "770", "missing": "0"}, "updatedLabel": "TaxonomyID"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "SourceKey58"}], "feature": "SourceKey58", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "CO109", "max": "PR96", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SourceKey58"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount59", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3042", "mean": "233.29", "stddev": "573.23", "min": "4.0", "max": "23900.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount59"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "ReasonDate"}], "feature": "ReasonDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "01/05/2020", "max": "31/03/2020", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "1.0", "stddev": "0.06", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "PracticeID"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "FacilityID63"}], "feature": "FacilityID63", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "087613d0-c7f3-4bcd-900a-abbc6ee3bdad", "max": "fde71f17-372e-4cde-95a7-d8b9b45a015a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityID63"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID66", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "1733.23", "stddev": "1207.04", "min": "1", "max": "4079", "missing": "0"}, "updatedLabel": "CPTCodeID66"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "DepartmentID"}], "feature": "DepartmentID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3042", "mean": "", "stddev": "", "min": "0a414089-7b3a-4378-ba12-23a2d7272bcf", "max": "fb68b47d-bad4-42c2-9ad0-9816864ea30a", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3042", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run xgboostHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(manualfe, ["ChargeDetailID0_stringindexer", "Amount1", "Amount2", "CPTCodeID3", "DiagnosisCodeID2", "DiagnosisCodeID1", "RVUModifier_stringindexer", "Gender_stringindexer", "EnterpriseID18", "PayerName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "SpecialityID", "TaxonomyID", "SourceKey58_stringindexer", "Amount59", "ReasonDate_stringindexer", "PracticeID", "FacilityID63_stringindexer", "CPTCodeID66", "DepartmentID_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
